In [2]:
anserini_root = 'D:/python/anserini' 
import os, sys
sys.path += [os.path.join(anserini_root, 'src/main/python')]

In [28]:
from pyserini.search import SimpleSearcher

#searcher = SimpleSearcher('indexes/index-robust04-20191213/')
searcher = SimpleSearcher('D:/python/anserini/lucene-index.robust04.pos+docvectors+rawdocs')
hits = searcher.search('hubble space telescope')

# Print the first 10 hits:
print(f"   {'Doc ID':15} Score")
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')
    
searcher.set_bm25(0.9, 0.4)
searcher.set_rm3(10, 10, 0.5,rm3_output_query=True)

hits2 = searcher.search('hubble space telescope')

# Print the first 10 hits:
for i in range(0, 10):
    print(f'{i+1:2} {hits2[i].docid:15} {hits2[i].score:.5f}')
    
searcher.unset_rm3()

   Doc ID          Score
 1 LA071090-0047   16.85690
 2 FT934-5418      16.75630
 3 FT921-7107      16.68290
 4 LA052890-0021   16.37390
 5 LA070990-0052   16.36460
 6 LA062990-0180   16.19260
 7 LA070890-0154   16.15610
 8 FT934-2516      16.08950
 9 LA041090-0148   16.08810
10 FT944-128       16.01920
 1 FT934-5418      5.24090
 2 LA071090-0047   5.19570
 3 LA070990-0052   4.94740
 4 LA052890-0021   4.93000
 5 FT921-7107      4.91800
 6 LA062990-0180   4.89580
 7 FT934-2516      4.80640
 8 LA070890-0154   4.78170
 9 LA041490-0064   4.74820
10 LA040190-0178   4.73550


In [29]:
from pyserini.search import querybuilder

# First, create term queries for each individual query term:
term1 = querybuilder.get_term_query('hubbl')
term2 = querybuilder.get_term_query('space')
term3 = querybuilder.get_term_query('telescop')

term4 = querybuilder.get_term_query('astronom')
term5 = querybuilder.get_term_query('univers')
term6 = querybuilder.get_term_query('light')
term7 = querybuilder.get_term_query('nasa')
term8 = querybuilder.get_term_query('mission')
term9 = querybuilder.get_term_query('optic')
term10 = querybuilder.get_term_query('mirror')

# Then, assemble into a "bag of words" query:
should = querybuilder.JBooleanClauseOccur['should'].value

boolean_query_builder = querybuilder.get_boolean_query_builder()
boolean_query_builder.add(term1, should)
boolean_query_builder.add(term2, should)
boolean_query_builder.add(term3, should)

query = boolean_query_builder.build()

hits = searcher.search(query)

for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA071090-0047   16.85690
 2 FT934-5418      16.75630
 3 FT921-7107      16.68290
 4 LA052890-0021   16.37390
 5 LA070990-0052   16.36460
 6 LA062990-0180   16.19260
 7 LA070890-0154   16.15610
 8 FT934-2516      16.08950
 9 LA041090-0148   16.08810
10 FT944-128       16.01920


In [30]:
boost1 = querybuilder.get_boost_query(term1, 0.26002878)
boost2 = querybuilder.get_boost_query(term2, 0.23582985)
boost3 = querybuilder.get_boost_query(term3, 0.27261537)
boost4 = querybuilder.get_boost_query(term4, 0.026429813)
boost5 = querybuilder.get_boost_query(term5, 0.027410464)
boost6 = querybuilder.get_boost_query(term6, 0.022568017)
boost7 = querybuilder.get_boost_query(term7, 0.06740909)
boost8 = querybuilder.get_boost_query(term8, 0.03151296)
boost9 = querybuilder.get_boost_query(term9, 0.020211682)
boost10 = querybuilder.get_boost_query(term10, 0.03598397)

should = querybuilder.JBooleanClauseOccur['should'].value

boolean_query_builder = querybuilder.get_boolean_query_builder()
boolean_query_builder.add(boost1, should)
boolean_query_builder.add(boost2, should)
boolean_query_builder.add(boost3, should)
boolean_query_builder.add(boost4, should)
boolean_query_builder.add(boost5, should)
boolean_query_builder.add(boost6, should)
boolean_query_builder.add(boost7, should)
boolean_query_builder.add(boost8, should)
boolean_query_builder.add(boost9, should)
boolean_query_builder.add(boost10, should)

query = boolean_query_builder.build()

hits = searcher.search(query)

for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 FT934-5418      5.19280
 2 LA071090-0047   5.16030
 3 LA070990-0052   4.94740
 4 LA062990-0180   4.89580
 5 LA052890-0021   4.86860
 6 FT921-7107      4.85180
 7 LA070890-0154   4.78170
 8 FT934-2516      4.75940
 9 LA040190-0178   4.73550
10 LA041490-0064   4.70390


<hr>

In [121]:
import pandas as pd

path = "D:/python/anserini/src/main/resources/topics-and-qrels/"
topics_path = "topics.dl19-passage.txt"
qrels_path = "qrels.dl19-passage.txt"

topics = pd.read_csv(path+topics_path,sep="\t",names=["ID","Question"])
topics["ID"]= topics["ID"].astype(str)

qrels = pd.read_csv(path+qrels_path,sep=" ",names=["Topic ID","","Doc ID","Grade"])
qrels = qrels.astype({"Topic ID": str, "Doc ID": str})
qrels["Topic ID"] = qrels["Topic ID"].astype(str)
qrels["Doc ID"] = qrels["Doc ID"].astype(str)

qrels.head()

,Topic ID,,Doc ID,Grade
0,19335,Q0,1017759,0
1,19335,Q0,1082489,0
2,19335,Q0,109063,0
3,19335,Q0,1160863,0
4,19335,Q0,1160871,0


In [69]:
topics.head()

,ID,Question
0,156493,do goldfish grow
1,1110199,what is wifi vs bluetooth
2,1063750,why did the us volunterilay enter ww1
3,130510,definition declaratory judgment
4,489204,right pelvic pain causes


In [175]:
from pyserini.search import SimpleSearcher

#searcher = SimpleSearcher('indexes/index-robust04-20191213/')
searcher = SimpleSearcher('D:/python/anserini/indexes/index-msmarco-passage-20191117-0ed488')
hits = searcher.search('do goldfish grow')

searcher.set_bm25(0.9, 0.4)
searcher.set_rm3(10, 10, 0.5,rm3_output_query=True)

k = 25
topicrow = 0
topicid = topics.iloc[topicrow]["ID"]
question = topics.iloc[topicrow]["Question"]
hits = searcher.search(question, k=k)
ids = []

print("Question: "+question)

print(f"{'#':2} {'ID':15} Score\tRel\tContent")
# Print the first 10 hits:
for i in range(0, k):
    rel = qrels.loc[(qrels['Doc ID'] == hits[i].docid) & (qrels['Topic ID'] == topicid)]["Grade"].values
    if rel.size == 0:
        continue
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}\t{rel[0]}\t{searcher.doc(hits[i].docid).raw()[:150]}...')
    ids.append(hits[i].docid)
searcher.unset_rm3()



Question: do goldfish grow
#  ID              Score	Rel	Content
 1 3288600         1.98930	2	A goldfish will grow to the depth of the water it is kept in. If the tank is not large enough to allow the goldfish to grow, it's growth will be stunt...
 2 8182166         1.97730	2	How big do goldfish grow? Are Koi considered big gold fish? I saw a photo of a goldfish that LOOKED like a goldfish, found in a lake. It was like 2-1/...
 3 3288596         1.96860	2	If the tank is not large enough to allow the goldfish to grow, it's growth will be stunted and this can cause the fish to become unhealthy and die. Th...
 4 1960255         1.87810	2	Rating Newest Oldest. Best Answer: Goldfish do not grow to the size of their tank, that is a myth. Goldfish grow until they are at their mature size, ...
 5 2612492         1.86470	2	The genetics (DNA) of the goldfish parents passed on to their offspring. How fast do goldfish grow is really a tough question to answer really. As men...
 6 2411918         1.

In [202]:
positive = qrels[(qrels['Topic ID'] == topicid) & (qrels['Doc ID'].isin(ids))]
negative = qrels[(qrels['Topic ID'] == topicid) & (~qrels['Doc ID'].isin(ids))]

In [210]:
tp = positive[positive['Grade']>0].shape[0]
fp = positive[positive['Grade']==0].shape[0]
fn = negative[negative['Grade']>0].shape[0]
tn = negative[negative['Grade']==0].shape[0]

In [219]:
precision = tp/(tp+fp)
precision

1.0

In [220]:
recall = tp/(tp+fn)
recall

0.18045112781954886

In [222]:
f1 = 2 * precision * recall / (precision+recall)
f1

0.3057324840764331

In [207]:
total = qrels[(qrels['Topic ID'] == topicid)]
total

,Topic ID,,Doc ID,Grade
1502,156493,Q0,1034435,1
1503,156493,Q0,1101607,2
1504,156493,Q0,1113693,0
1505,156493,Q0,1171591,0
1506,156493,Q0,117188,0
...,...,...,...,...
1797,156493,Q0,95519,2
1798,156493,Q0,95521,2
1799,156493,Q0,994482,0
1800,156493,Q0,994486,2
